# Filling in missing data of CO$_2$ time series.

This notebook fills in the missing values using ETS models and creates new .csv files with the updated information. 

## Libraries

In [2]:
import numpy as np
import pandas as pd
import glob
import ast
import warnings
import statsmodels.api as sm
from math import sqrt
from datetime import *
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from numpy import array
from sklearn import metrics
from sklearn.model_selection import ParameterGrid


## Paths

In [3]:
pathr='/home/_ehoyos/Documents/Data_CO2/Final_info/' # path to read the data.
paths='/home/_ehoyos/Documents/Data_CO2/Final_results/' # path to save the results.

### Definition of functions 

In [46]:
### Functions to select the best model

def exp_smoothing_forecast(history,config):
    t,d,s,p,r=config
    # define model
    history=array(history)
    model=ExponentialSmoothing(history,trend=t,damped_trend=d,seasonal=s,seasonal_periods=p)
    # fit model
    model_fit=model.fit(optimized=True,remove_bias=r)
    yhat=model_fit.predict(0,len(history)-1)  
    # estimate prediction error
    error,r2=measure_rmse(history,yhat)
    return (error,r2,model_fit.params["smoothing_level"],model_fit.params["smoothing_trend"],
           model_fit.params["smoothing_seasonal"],model_fit.params["damping_trend"])

# root mean squared error or rmse
def measure_rmse(actual, predicted):
    error=sqrt(mean_squared_error(actual, predicted))
    r2=metrics.r2_score(actual,predicted) 
    return (error,r2)

# split a univariate dataset into train/test sets
def train_test_split(dataf,n_test):
    return dataf,dataf 

# walk-forward validation for univariate data
def walk_forward_validation(dataf,n_test,cfg):
    # split dataset
    train,test=train_test_split(dataf,n_test)
    # seed history with training dataset
    history=[x for x in train]
    error,r2,a1,b1,g1,f1=exp_smoothing_forecast(history,cfg) 
    return (error,r2,a1,b1,g1,f1)

# score a model, return None on failure
def score_model(dataf,n_test,cfg,debug=False):
    result=None
    r2=None
    aa=None;bb=None;gg=None;ff=None #values of alpha, beta, gamma and phi.
    # convert config to a key
    key=str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result,r2,aa,bb,gg,ff=walk_forward_validation(dataf,n_test,cfg)
    else:
        # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                result,r2,aa,bb,gg,ff=walk_forward_validation(dataf,n_test,cfg)
        except:
            error=None
    return (key,result,r2,aa,bb,gg,ff)

# grid search configs
def grid_search(dataf,cfg_list,n_test,parallel=True):
    scores=None
    if parallel:
        # execute configs in parallel
        executor=Parallel(n_jobs=cpu_count(),backend='multiprocessing')
        tasks=(delayed(score_model)(dataf,n_test,cfg) for cfg in cfg_list)
        scores=executor(tasks)
    else:
        scores=[score_model(dataf,n_test,cfg) for cfg in cfg_list]
    # remove empty results
    scores=[r for r in scores if r[1] !=None]
    # sort configs by error, asc
    scores.sort(key=lambda tup:tup[1])
    return scores

# create a set of exponential smoothing configs to try
def exp_smoothing_configs(seasonal=[None]):
    models=list()
    # define config lists
    t_params=['add','mul',None] # trend.
    d_params=[True,False] # damped trend.
    s_params=['add','mul',None] # seasonal.
    p_params=seasonal # seasonal period.
    r_params=[True,False] # remove bias.
    # create config instances
    for t in t_params:
        for d in d_params:
            for s in s_params:
                for p in p_params:
                    for r in r_params:
                        cfg=[t,d,s,p,r]
                        models.append(cfg)
    return models

### Functions to fill in the missing values after selecting the best model

def exp_smoothing_forecast_f(history,config):
    t,d,s,p,r=config
    history=array(history)
    model=ExponentialSmoothing(history,trend=t,damped_trend=d,seasonal=s,seasonal_periods=p)
    model_fit=model.fit(optimized=True,remove_bias=r)
    yhat=model_fit.predict(len(history),len(history))
    return yhat[0]

def train_test_split_f(dataf,n_test):
    return dataf[:-n_test],dataf[-n_test:] 

def walk_forward_validation_f(dataf,n_test,cfg):
    predictions=list()
    train,test=train_test_split_f(dataf,n_test)
    history=[x for x in train]
    for i in range(n_test): 
        yhat=exp_smoothing_forecast_f(history,cfg)
        warnings.filterwarnings('ignore')
        predictions.append(yhat)
        history.append(yhat) 
    return history

def ci_additive(dataf,config,n_test):
    t,d,s,p,r=config
    ci_lower=list(); ci_upper=list()
    ets_model=sm.tsa.statespace.ExponentialSmoothing(dataf,trend=t,damped_trend=d,seasonal=p).fit()
    results=ets_model.get_forecast(int(n_test))
    return results

### Function to define the start and end of consecutive missing data

def get_nan_inds(series):
    series=series.reset_index(drop=True)
    index=series[series.isna()].index.to_numpy()
    if len(index)==0:
        return []
    indices=np.split(index,np.where(np.diff(index)>1)[0]+1)
    return [(ind[0],ind[-1]+1) for ind in indices]

### Procedure

In [1]:
files=glob.glob(pathr+"/*.csv")
names=["" for i in range(len(files))] # names of sites.
for i in range(len(files)):
    names[i]=files[i].split('/')[len(files[i].split('/'))-1].split('.')[0]

for ii in range(len(names)):
#for ii in range(0,1):
    info_db=[] # dataframe of statistic info of nan filling.
    dci_db=[]  # dataframe of confidence intervals info of nan filling.
    print('i='+str(ii),names[ii])
    start_run=datetime.now()
    w=names.index(names[ii])
    data=pd.read_csv(files[w],skiprows=11)
    line=open(files[w], "r").readlines()[0:8]
    site=line[0].split(',')[1].strip()
    code=line[1].split(',')[1].strip()
    country=line[2].split(',')[1].strip()
    latitude=line[3].split(',')[1].strip()
    longitude=line[4].split(',')[1].strip()
    altitude=line[5].split(',')[1].strip()
    units=line[6].split(',')[1].strip()
    nHeights=int(line[7].split(',')[1].strip())
    
    titles=data.columns.values.tolist()
    titles2=data.columns[7:7+nHeights] # columns of data for each height.
    data["date"]=pd.to_datetime(data["date"])
    data['doy']=data['date'].dt.dayofyear
    
### Replace missing data with NaN
    data=data.replace(-999.0,np.NaN)
    data=data.replace(-999.99,np.NaN)
    for i in range(nHeights):
        data[titles2[i]][data[titles2[i]]<0]=np.NaN
    
### Identify time resolution
    w=data['date'][1].minute-data['date'][0].minute
    if w==30:
        dt=0.5 #delta of time [d].
    elif w==0:
        dt=1
    else:
        print('dt is not 1 h or 0.5 h')
    
    maxf=24/dt # maximum length to fill (1 day=48 or 24 values). ***
    minb=3*24/dt # minimum length of values before the missing value (3 days=144 or 72). ***
    
    p_miss0=np.zeros(nHeights); p_missf=np.zeros(nHeights) # percentage of missing data before and after filling.
    for jj in range(nHeights):
    #for jj in range(1):
        count=0;count2=0
        print(titles2[jj])
        series=data[titles2[jj]]
        
### Define the start and end of consecutive missing data
        nan_index=np.array(get_nan_inds(series)) # start and end of consective nan data.
        nan_indexf=nan_index # vector with consecutive nan data positions that are not filled.
        if nan_index[0][0]!=0: 
            nan_indexf=np.insert(nan_indexf,0,[[0]],axis=0)
        series=np.array(series)
        info=[] # array to write the statistics of the procedure to fill in the missing data.
        dci=[] # array to write the amplitude of the confidence interval and the mean SE.
        
        for i in range(nan_index.shape[0]):
            lenf=nan_index[i][1]-nan_index[i][0]
            #print(lenf)
            if lenf<=maxf:
                if nan_index[i][0]!=0: # if there are values before.
                    w=np.where([nan_indexf[:,1]<nan_index[i][0]])[1].max()
                    lenb=nan_index[i][0]-nan_indexf[w,1]
                    print(lenb)
                    if lenb>minb:
                        count=count+1
                        print(f"{i}/{nan_index.shape[0]}_{names[ii]}")
                        w2=np.where([nan_indexf==nan_index[i]])[1][0]
                        nan_indexf=np.delete(nan_indexf,w2,0)
                        dataf=series[nan_indexf[w,1]:nan_index[i][0]]
### Define the best model describing the previous part of the series.                
                        n_test=lenf
                        cfg_list=exp_smoothing_configs(seasonal=[24/dt])
                        scores=grid_search(dataf,cfg_list,n_test)
                        for cfg,error,r2,aa,bb,gg,ff in scores[:1]:
                            print(cfg,error,r2,aa,bb,gg,ff) 
                            info_fill=[i,nan_indexf[w,1],nan_index[i][0],lenf,error,r2]
                            info.append(info_fill)
                        
### Using the best model, fill the missing data.  
                        cfg=ast.literal_eval(scores[0][0])
                        dataf2=series[nan_indexf[w,1]:nan_index[i][1]]
                        fill=np.array(walk_forward_validation_f(dataf2,n_test,cfg))
                        series[nan_index[i][0]:nan_index[i][1]]=fill[fill.size-n_test:] 
### If the model is additive, it finds the confidence interval for each filled value.
                        if cfg[0]!='mul' and cfg[2]!='mul':
                            count2=count2+1
                            dataf2=series[nan_indexf[w,1]:nan_index[i][1]]
                            confidence=ci_additive(dataf2,cfg,n_test)
                            d_ci=confidence.summary_frame(alpha=0.05)[["mean"]].values-confidence.summary_frame(alpha=0.05)[["mean_ci_lower"]].values # amplitude of the confidence interval.
                            mean_se=confidence.summary_frame(alpha=0.05)[["mean_se"]].values
                            for l in range(n_test):
                                x=[i,float(d_ci[l]),float(mean_se[l])]
                                dci.append(x)
        if jj==0:
            dci_db=pd.DataFrame(dci,columns=['id_'+titles2[jj],'dci_'+titles2[jj],
                                                                    'se_'+titles2[jj]])
        else:
            df=pd.DataFrame(dci,columns=['id_'+titles2[jj],'dci_'+titles2[jj],
                    'se_'+titles2[jj]])
        if jj>0:
            dci_db=pd.concat([dci_db,df],ignore_index=False,axis=1)                                
### Save statistics in a dataframe 
# initial (t0) and final (tf) times considered to select the best model
# Lnan is the length of the consecutive nan data filled, MSE is the error, r2)        
        if count>0:
            if jj==0:
                info_db=pd.DataFrame(info,
                columns=['id_'+titles2[jj],'t0_'+titles2[jj],'tf_'+titles2[jj],'Lnan_'+titles2[jj],
                         'MSE_'+titles2[jj],'r2_'+titles2[jj]])
            else:
                df2=pd.DataFrame(info,
                columns=['id_'+titles2[jj],'t0_'+titles2[jj],'tf_'+titles2[jj],'Lnan_'+titles2[jj],
                         'MSE_'+titles2[jj],'r2_'+titles2[jj]])
            if jj>0:
                info_db=pd.concat([info_db,df2],ignore_index=False,axis=1)    
### Save filled data in a new column of the dataframe
        data[titles2[jj]+"_fill"]=series
        p_miss0[jj]=data[titles2[jj]].isnull().sum()/data.shape[0]*100
        p_missf[jj]=data[titles2[jj]+"_fill"].isnull().sum()/data.shape[0]*100
        print(p_miss0[jj],p_missf[jj])
### Save the filled dataframe in a new file.
    header='Site,'+site+'\nCode,'+code+'\nCountry,'+country+'\nLatitude,'+str(latitude)+'\nLongitude,'+str(longitude)+'\nAltitude,'+str(altitude)+'\nUnits,'+units+'\nnHeights,'+str(nHeights)+'\n'+'\n'
    with open(paths+code+'_filled.csv', 'w') as fp:
        fp.write(header)
    data.to_csv(paths+code+'_filled.csv',header=True,mode='a')            
### Save the statistics info dataframe in a new file.
    if len(info_db)!=0:
        info_db=info_db.append({'id_'+titles2[0]:'' },ignore_index=True)
        info_db=info_db.append({'id_'+titles2[0]:'Height','t0_'+
                            titles2[0]:'p before','tf_'+titles2[0]:'p after'},ignore_index=True)
        for l in range(nHeights):
            info_db=info_db.append({'id_'+titles2[0]:titles2[l],'t0_'+titles2[0]:p_miss0[l],
                                'tf_'+titles2[0]:p_missf[l]},ignore_index=True)
        info_db.to_csv(paths+code+'_filled_info.csv',header=True)    
### Save the confidence interval info dataframe in a new file.
    if len(dci_db)!=0:
        dci_db.to_csv(paths+code+'_filled_ci.csv',header=True) 
    end_run=datetime.now()   
    print(f"Runtime of the program is {end_run-start_run}") 
        
        
# *** values to define.